In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.0 MB/s eta 0:00:0000:01


In [3]:
from torch_geometric.datasets import Planetoid, Amazon, Reddit, WikiCS, Flickr, WebKB, Actor, PolBlogs, CitationFull
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data
from torch_geometric.transforms import AddSelfLoops
from torch_geometric.utils import subgraph, k_hop_subgraph
import torch_geometric.transforms as T
import torch
import math
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch_geometric.nn import global_add_pool, global_max_pool, GlobalAttention, global_mean_pool
from torch_geometric.nn.inits import glorot




edge_index = torch.tensor([[0, 1, 0],
                           [1, 2, 0]], dtype=torch.long)
x = torch.tensor([[1,2,3],
                 [4,5,6],
                 [6,7,8]])
y = torch.tensor([0,1,2])
data = Data(x=x, edge_index=edge_index, y=y)
# 创建 AddSelfLoops 转换
add_self_loops = AddSelfLoops()
# 应用转换
data = add_self_loops(data)
print(data.edge_index)
data.edge_index = torch.unique(data.edge_index, dim=1)
print(data.edge_index)

tensor([[0, 1, 0, 0, 1, 2],
        [1, 2, 0, 0, 1, 2]])
tensor([[0, 0, 1, 1, 2],
        [0, 1, 1, 2, 2]])


In [4]:
!pwd

/kaggle/working


In [5]:
def induced_graphs(data, device, smallest_size=10, largest_size=30):   # 构建诱导图的过程
    induced_graph_list = []
    from copy import deepcopy
    
    for index in range(data.x.size(0)):
        current_label = data.y[index].item()

        current_hop = 2
        subset, _, _, _ = k_hop_subgraph(node_idx=index, num_hops=current_hop,
                                            edge_index=data.edge_index, relabel_nodes=True)
        subset = subset

        while len(subset) < smallest_size and current_hop < 5:
            current_hop += 1
            subset, _, _, _ = k_hop_subgraph(node_idx=index, num_hops=current_hop,
                                                edge_index=data.edge_index)
            
        if len(subset) < smallest_size:
            need_node_num = smallest_size - len(subset)
            pos_nodes = torch.argwhere(data.y == int(current_label))   # Test data may leak
            pos_nodes = pos_nodes.to('cpu')
            subset = subset.to('cpu')
            candidate_nodes = torch.from_numpy(np.setdiff1d(pos_nodes.numpy(), subset.numpy()))
            candidate_nodes = candidate_nodes[torch.randperm(candidate_nodes.shape[0])][0:need_node_num]
            subset = torch.cat([torch.flatten(subset), torch.flatten(candidate_nodes)])

        if len(subset) > largest_size:
            subset = subset[torch.randperm(subset.shape[0])][0:largest_size - 1]
            subset = torch.unique(torch.cat([torch.LongTensor([index]).to(device), torch.flatten(subset).to(device)]))

        subset = subset.to(device)
        sub_edge_index, _ = subgraph(subset, data.edge_index, relabel_nodes=True)
        sub_edge_index = sub_edge_index.to(device)

        x = data.x[subset]

        induced_graph = Data(x=x, edge_index=sub_edge_index, y=data.y[index], index = index)
        add_self_loops = AddSelfLoops()
        induced_graph = add_self_loops(induced_graph)
        induced_graph.edge_index = torch.unique(induced_graph.edge_index, dim=1)
        induced_graph_list.append(induced_graph)
        if index%500 == 0:
            print(index)
    return induced_graph_list


transform_list = [T.AddSelfLoops(), T.ToUndirected(), T.NormalizeFeatures()]
transform = T.Compose(transform_list)
dataset = Planetoid(root='/kaggle/working', name='Cora', transform=transform)
data = dataset[0]
graph_list = induced_graphs(data, 'cpu')
print(graph_list[0],graph_list[0].index)

Processing...
Done!


0
500
1000
1500
2000
2500
Data(x=[29, 1433], edge_index=[2, 47], y=3, index=0) 0


In [6]:
def normalize_adj_tensor(adj):
    # gcn的归一化邻接矩阵方法
    D = torch.sum(adj, dim=1)
    D_inv = torch.pow(D, -1 / 2)
    D_inv[torch.isinf(D_inv)] = 0.
    D_mat_inv = torch.diag(D_inv)
    adj_norm = D_mat_inv @ adj @ D_mat_inv  # GCN的归一化方式
    return adj_norm

def edge_index_to_adjacency_matrix(edge_index, num_nodes, undirected=True, device='cpu'):  
    # 将edge_indedx转化为邻接矩阵
    # 构建一个大小为 (num_nodes, num_nodes) 的零矩阵  
    adjacency_matrix = torch.zeros(num_nodes, num_nodes, dtype=torch.uint8).to(device)
      
    # 使用索引广播机制，一次性将边索引映射到邻接矩阵的相应位置上  
    if undirected:
        adjacency_matrix[edge_index[0], edge_index[1]] = 1  
        adjacency_matrix[edge_index[1], edge_index[0]] = 1  
    else:
        adjacency_matrix[edge_index[0], edge_index[1]] = 1
    return adjacency_matrix


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
size_buffer = []
for graph in graph_list:
    size_buffer.append(graph.x.shape[0])
    adj = edge_index_to_adjacency_matrix(graph.edge_index, graph.x.shape[0])
    graph.adj = adj

print(graph_list[0])


Data(x=[29, 1433], edge_index=[2, 47], y=3, index=0, adj=[29, 29])


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Batch

class GraphDataset(Dataset):
    def __init__(self, graph_list):
        self.graph_list = graph_list

    def __len__(self):
        return len(self.graph_list)

    def __getitem__(self, idx):
        return self.graph_list[idx]

class SequentialGraphLoader(DataLoader):
    def __init__(self, graph_list, batch_size=1):
        dataset = GraphDataset(graph_list)
        super(SequentialGraphLoader, self).__init__(dataset, batch_size=batch_size, shuffle=False)

    def __iter__(self):
        for i in range(0, len(self.dataset), self.batch_size):
            batch = self.dataset[i:i+self.batch_size]
            yield self.merge_graphs(batch)

    def merge_graphs(self, batch):
        # 合并子图的特征矩阵和邻接矩阵
        x_buffer = []
        adj_list = []
        y_buffer = []
        batch_sizes = []
        batch_indexs = []
        graph_indexs = []

        for i, graph in enumerate(batch):
            x = graph.x
            adj = graph.adj
            y = graph.y.unsqueeze(0)  # 确保 y 是二维张量
            x_buffer.append(x)
            adj_list.append(adj)
            y_buffer.append(y)
            batch_sizes.append(x.shape[0])  # 存储子图的大小
            batch_indexs = batch_indexs + [i] * x.shape[0]
            graph_indexs.append(graph.index)

        # 将特征矩阵堆叠
        x_combined = torch.cat(x_buffer, dim=0)  # 合并特征矩阵
        num_nodes = x_combined.size(0)  # 所有节点的总数
        
        # 构建大的邻接矩阵
        adj_combined = torch.zeros(num_nodes, num_nodes, device=x_combined.device)
        start = 0
        for i in range(len(batch)):
            end = start + batch_sizes[i]
            adj_combined[start:end, start:end] = adj_list[i]  # 填充对应的子图邻接矩阵
            start = end

        y_combined = torch.cat(y_buffer, dim=0)  # 合并标签
        batch_indexs = torch.tensor(batch_indexs)

        return x_combined, adj_combined, y_combined, batch_sizes, batch_indexs, graph_indexs

In [8]:
from torch_geometric.data import Batch, Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool
import math
import copy

modified_graph_list = copy.deepcopy(graph_list)
adj_changes = [torch.nn.Parameter(torch.FloatTensor(size, size)) for size in size_buffer] # 对每个子图的扰动
for adj_change in adj_changes:
    adj_change.data.fill_(0)
for modified_graph, graph, adj_change in zip(modified_graph_list, graph_list, adj_changes):
    change_square = adj_change - torch.diag(torch.diag(adj_change, 0))
    change_square = torch.clamp(change_square, -1, 1)
    modified_graph.adj = change_square + graph.adj

train_loader = SequentialGraphLoader(modified_graph_list[:800], batch_size=16) # 定义的Dataloader
test_loader = SequentialGraphLoader(modified_graph_list[800:], batch_size=16)
weights = []
w_velocities = []
hidden_sizes = [128 for i in range(2)]
previous_size = 1433
out_dim = 7
for ix, nhid in enumerate(hidden_sizes):
    weight = torch.nn.Parameter(torch.FloatTensor(previous_size, nhid).to(device))
    w_velocity = torch.zeros(weight.shape).to(device)
    weights.append(weight)
    w_velocities.append(w_velocity)
    previous_size = nhid
    
output_weight = torch.nn.Parameter(torch.FloatTensor(previous_size, out_dim).to(device))
output_w_velocity = torch.zeros(output_weight.shape).to(device)
weights.append(output_weight)
w_velocities.append(output_w_velocity)
for w, v in zip(weights, w_velocities):
    stdv = 1. / math.sqrt(w.size(1))
    w.data.uniform_(-stdv, stdv)
    v.data.fill_(0)

criterion = nn.CrossEntropyLoss()  # 分类任务的损失函数


#初始化参数,避免梯度爆炸
for w, v in zip(weights, w_velocities):
    stdv = 1. / math.sqrt(w.size(1))
    w.data.uniform_(-stdv, stdv)
    v.data.fill_(0)


for epoch in range(10):
    print("epoch_num:", epoch)
    for x, adj, y, sizes, batch_indexs, graph_indexs in train_loader:
        loss = 0.0
        x, adj, y, batch_indexs = x.to(device), adj.to(device), y.to(device), batch_indexs.to(device)
        adj_norm = normalize_adj_tensor(adj)
        
        # 图卷积
        for i, w in enumerate(weights):
            if i != len(weights)-1:
                x = adj_norm @ x @ w
            else:
                x = global_mean_pool(x, batch_indexs) @ w
        
        # 计算损失
        loss = criterion(x, y)
        assert torch.isnan(loss).any()==False, "loss is NaN"
        if torch.isnan(loss).any():
            raise ValueError("loss is NaN!")
        weight_grads = torch.autograd.grad(loss, weights, create_graph=True)
        w_velocities = [0.9 * v + g for v, g in zip(w_velocities, weight_grads)]
        weights = [w - 0.01 * v for w, v in zip(weights, w_velocities)]
        
total_loss = 0.0
for x, adj, y, sizes, batch_indexs, graph_indexs in test_loader:
    x, adj, y, batch_indexs = x.to(device), adj.to(device), y.to(device), batch_indexs.to(device)
    adj_norm = normalize_adj_tensor(adj)
    
    # 图卷积
    for i, w in enumerate(weights):
        if i != len(weights)-1:
            x = adj_norm @ x @ w
        else:
            x = global_mean_pool(x, batch_indexs) @ w
        
    # 累计损失
    total_loss += criterion(x, y) * x.shape[0]

total_loss.backward(retain_graph=False)

# 提取元梯度
adj_grads = [adj_change.grad for adj_change in adj_changes]

# 验证形状
for grad, adj_change in zip(adj_grads, adj_changes):
    assert grad.shape == adj_change.shape
print("done")
print(adj_grads[0])

epoch_num: 0
epoch_num: 1
epoch_num: 2
epoch_num: 3
epoch_num: 4
epoch_num: 5
epoch_num: 6
epoch_num: 7
epoch_num: 8
epoch_num: 9
done
tensor([[ 0.0000e+00,  1.5383e-01,  1.0776e-01,  1.8358e-01,  1.1978e-01,
          2.1887e-01,  1.4241e-01,  1.0948e-01,  9.0374e-05,  1.1781e-01,
          1.5431e-01,  1.1072e-01,  3.7291e-02,  1.1588e-01,  1.3037e-01,
          3.5654e-02,  7.6392e-02,  1.1063e-01,  3.5926e-02,  4.7406e-02,
          1.4259e-01,  3.0767e-02,  9.3560e-02,  1.5020e-01,  1.5903e-01,
          1.2106e-01,  8.3091e-02,  4.8372e-02,  5.1876e-02],
        [-1.5383e-01,  0.0000e+00, -4.6078e-02,  2.9744e-02, -3.4050e-02,
          6.5039e-02, -1.1426e-02, -4.4355e-02, -1.5374e-01, -3.6026e-02,
          4.7378e-04, -4.3112e-02, -1.1654e-01, -3.7951e-02, -2.3465e-02,
         -1.1818e-01, -7.7441e-02, -4.3204e-02, -1.1791e-01, -1.0643e-01,
         -1.1244e-02, -1.2307e-01, -6.0273e-02, -3.6296e-03,  5.1948e-03,
         -3.2769e-02, -7.0742e-02, -1.0546e-01, -1.0196e-01],
 

In [9]:
#在上述实验中可以发现需要较小的batch_size以支持meta-gradient的计算(e.g., batch_size=16); 如果太大则会出现Out-of-Memory, 因此在这里记录一下梯度图空间复杂度的理论分析：

#假设现在有m个子图，平均每个图有n个节点，那么对于一个epoch来说，有m/batch_size个batch;

#由于需要拼接整个batch的子图成一个大图，那么这个大图的邻接矩阵规模为 (n * batch_size)^2

#综上, 存储的梯度图理论空间复杂度为：
#O(m/batch_size*(n*batch_size)^2) = O(m*n^2*batch_size)

#进一步考虑到完整的surrogate_training有k个epoch,那么整个surrogate_training的空间复杂度为：
#O(k*m*n^2*batch_size)

#因此过大的batch_size必然导致过大的内存开销
#但如果batch_size过小, 比如极端情况下batch_size=1, 那么理论上讲导致surrogate_training过程过于“琐碎”, 无法感知到训练的全局方向。（TODO, 需要进一步做一个实验）
#上面问题更新:目前部分实验做下来batch_size不会特别影响surrogate_training的效果,但batch_size过小将导致更长的开销时间（TODO, 再进一步做一个实验）

#现在分析node_injection的理论空间复杂度:
#进一步假设注入的节点个数为原图个数的b%,那么一共有m*n*b%个注入节点,每个注入节点只考虑添加到一个子图,因此优化节点注入位置的拓扑有n条,那么需要元梯度的拓扑扰动总数为m*n^2*b%
#如果同时也考虑优化节点的d维特征,那么需要元梯度的特征总数为m*n*d*b%
#整个surrogate_training的空间复杂度为:
#O(k*m*n*(n+d)*b%)

#分析O(k*m*n^2*batch_size)和O(k*m*n*(n+d)*b%)大小, 即比较n*batch_size和（n+d）*b%
#往往为了不同任务域迁移,需要PCA降维,d不会过大，如100维
#因此显然 n*batch_size >>（n+d）*b%
